In [2]:
!pip install eli5

     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import  cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import  literal_eval
from tqdm import tqdm_notebook

In [5]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [6]:
ls

data/  day4.ipynb  HelloGithub.ipynb  LICENSE  matrix_one/  README.md


In [7]:
ls data

men_shoes.csv  shoes_prices.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [10]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [16]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])


(-58.133398968282776, 4.206122611474276)

In [17]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan' : return output_dict

  features = literal_eval(x.replace('\\"','"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value
  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [27]:
keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys()))

len(keys)

476

In [28]:
def get_name_feat(key):
  return 'feat_' + key


for key in tqdm_notebook(keys):  
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] *100



In [38]:
{k: v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_brand_gender'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]


In [42]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df[df.brand == df.feat_brand ].shape

(8846, 526)

In [46]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'],model)

(-57.27888225912316, 4.267106492005568)

In [0]:
feats_cat = [x for x in df.columns if 'cat' in x]

In [0]:
feats = ['brand_cat', 'feat_brand_cat','feat_brand_gender', 'feat_material_cat', 'feat_style_cat', 'feat_sport_cat' , 'feat_metal type_cat']
#feats += feats_cat
#feats = list(set(feats))

model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

In [64]:
X = df[ feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

(-57.21460169993011, 4.2596624276769965)


Weight,Feature
0.2594 ± 0.0110,brand_cat
0.1013 ± 0.0074,feat_material_cat
0.0406 ± 0.0025,feat_brand_gender
0.0179 ± 0.0009,feat_brand_cat
0.0093 ± 0.0010,feat_metal type_cat
0.0053 ± 0.0009,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [55]:
df[ df['brand'] == 'nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [67]:
!git add day5.ipynb
!git commit -m "Predykcja cen 2"

fatal: pathspec 'day5.ipynb' did not match any files
On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
	modified:   day4.ipynb
	modified:   matrix_one/day3.ipynb

no changes added to commit


In [0]:
!git config --global user.email "jarek@authoring.pl"
!git config --global user.name "wideowsieci"